# How to get details from RoboHive Environments
RoboHive supports three ways to query the environment based on the nature of information required. Following information can be queried from an active environment
1. Get observations (configured using `obs_keys`)
2. Get proprioception (configured via `proprio_keys`)
3. Get exteroception (configured via `visual_keys`)

Let's go through them in detail one at a time. First we load an environment, preconfigured with respective keys, and step it a few times -
(we will go through the key configuration details in another tutorial).

In [ ]:
import gym 
import robohive

# create an env and reset
env = gym.make('door_v2d-v1')
env.reset()

# Lets step is few times
for _ in range(5):
    allinfo_tdt = env.step(env.action_space.sample())

### 1. get observations
This is the most commonly used method to get the current (state based) observation from the environment. Observations are customized using `env.obs_keys` and can contain any details that the environment has access to. RoboHive doesn't put any restrictions on the information that can be provided by the environment.

RoboHive env provides an `env.get_obs()` method to query observations. It triggers the following sequence of events -
1. Uses a robot (sim/hardware) to get sensors
2. Reconstructs (partially) observed-sim `env.sim_obsd` using (noisy) sensor data
3. Build the full observation dictionary `env.obs_dict` using the observed-sim
4. Build obs vector from the obs_dict (using the `env.obs_keys`)

In [ ]:
# get current observation vector (internally updates env.sim_obsd & env.obs_dict)
obs = env.get_obs()

### 2. get proprioception
This method is used to get *only proprioceptive signals* from the robohive environments. Proprioceptive signals are customized using `env.proprio_keys` and typically consist of proprioceptive sensors that are available to the agent/robots. Proprioception can be acquired in two ways     
1. Recover from *existing* observation dictionary
2. Update observation and get proprioception alongside (default behavior)

In [ ]:
# Recover from existing observation dictionary 
time, proprio_vec, proprio_dict = env.get_proprioception(env.obs_dict)

# Update observation and get proprioception alongside
obs = env.get_obs(update_proprioception=True)
# you can get access proprioception dictionary now
print(env.proprio_dict)

### 3. get exteroception
This method is used to get *only exteroceptive signals* from the robohive environments. exteroceptive signals typically consist of exteroceptive sensors that are available to the agent/robots. Currently we are focusing primarily on cameras as exteroceptive inputs. It can be customized using `env.visual_keys`. Expanding to other exteroceptive modalities while possible and within scope, is not a tested functionality yet.

Note that exteroceptive sensors are expensive (compute + bandwidth), therefore its requires users to make explicit calls for an update. It can be acquired in two ways -

1. Make an explicit call to `env.get_exteroception()`
2. Update observation and get proprioception alongside (False by default)

In [ ]:
# Make an explicit call to update
extero_dict = env.get_exteroception()

# Update observation and get exteroception alongside
obs = env.get_obs(update_exteroception=True)
print(env.visual_dict.keys())

### 4. get everything
If it's of interest to get all the information at once, `env_get_obs()` can be asked to make all the updates.


In [ ]:
obs = env.get_obs(update_proprioception=True, update_exteroception=True)
print(f"time = {env.obs_dict['time']}")
print(f"obs vector = {obs}")
print(f"obs_dict = {env.obs_dict.keys()}")
print(f"proprio_dict = {env.proprio_dict.keys()}")
print(f"visual_dict = {env.visual_dict.keys()}")

# Practical tips

### 1. Why ['time', 'time'] as observation?
Note the observation vector - it looks too simple for such an environment of this complexity. This seems a little weird!
It is a common practice in *RoboHive* to use ['time', 'time'] as observation for envs designed for visual diversity. This is for two reasons 
1. To avoid leaking oracle information via obs to the agents studying visual generalization 
2. Single ['time'] key leads to silent singleton expansion when inputs of higher dimensions are required. Replicating the `time` keys twice helps to catch/expose this bug.

### 2. Ways to ask for obs, proprioception, exteroception?
The flexibility of RoboHive allows multiple ways to ask for information from the env. We outline a of options below - 

In [ ]:
# Recover all info at current timestep: obs(t), rwd(t), done(t), info(t)
obs_t, rwd_t, done_t, info_t = env.env.forward(update_proprioception=True, update_exteroception=True)
print(f"time = {env.obs_dict['time']}")
print(f"obs vector = {obs_t}")
print(f"obs_dict = {env.obs_dict.keys()}")
print(f"proprio_dict = {env.proprio_dict.keys()}")
print(f"visual_dict = {env.visual_dict.keys()}")

# Recover info at the next timestep: obs(t+dt), rwd(t+dt), done(t+dt), info(t+dt)
obs_tdt, rwd_tdt, done_tdt, info_tdt = env.env.step(env.action_space.sample(), update_proprioception=True, update_exteroception=True)
print(f"time = {env.obs_dict['time']}")
print(f"obs vector = {obs_tdt}")
print(f"obs_dict = {env.obs_dict.keys()}")
print(f"proprio_dict = {env.proprio_dict.keys()}")
print(f"visual_dict = {env.visual_dict.keys()}")

In [ ]:
# close the env
env.close()